In [8]:
import requests 
from bs4 import BeautifulSoup
from time import sleep
from time import time
import random
import sqlite3
import re
from datetime import datetime
import datetime as dt
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np

In [10]:
def isNaN(string):
    return string != string

carBrands = ["Audi", "BMW", "Citroen","Ferrari"]
#                ,"Fiat","Honda", "Hyundai", "Kia", "Land Rover","Lexus" 
#                "mazada", "mercedes", "MINI", "mitsubishi", "nissan", "peugeot", "porsche", "renault", "subaru", 
#                "suzuki", "toyota", "volkswagen", "volvo" ]

def scraping(webpage):
    
    dict_car = []
    price = []
    desc = []
    brands = []
    depreciationValue = []
    registrationDate = []
    category = []
    mileage = []
    images = []
    engineCapacity = []

    soup = BeautifulSoup(webpage)

    #find price
    for a in soup.find_all("td", class_="font_red"):
        a_clean = a.text.strip()[1:].replace(',','')
        try:
            int(a_clean)
            a_int = int(a_clean)
        except ValueError:
            a_int = 0
        price.append(a_int)

    # find car description
    for car in soup.find_all('div', attrs={'style':'width:186px;padding-left:4px;'}):
        #split into brand and car model
        brand = car.text.split()[0]
        brands.append(brand)
        description = car.text.replace(brand+' ','')
        desc.append(description)

    # find decpreciation value
    for depreciation in soup.find_all('div', attrs={'style':'width:101px;'}):
        depreciationText = depreciation.text.strip().split()[0].replace("$", "").replace(",","")
        depreciationValue.append(depreciationText)

    # find registration date and Vechicle Type    
    tempArray =[]

    for registrationAndType in soup.find_all('div', attrs={'style':'width:89px;'}):
        registrationAndType_clean = registrationAndType.text.strip()[:]
        tempArray.append(registrationAndType_clean)

    for i in range(0, len(tempArray)):
        #put into list of registration date
        if(i%2 == 0):
            registrationDate.append(tempArray[i])
        #put into list of category type
        else:
            category.append(tempArray[i])

    #find mileage
    for mileages in soup.find_all('div', attrs={'style':'width:83px;'}):
        mileageNumber = mileages.text.strip()[:-3].replace(",","")
        try:
            mile = int(mileageNumber)
        except ValueError:
            mile = 0
        mileage.append(mile)
        
    for engineCC in soup.find_all('div', attrs={'style':'width:84px;'}):
        engineCCText = engineCC.text.strip().split()[0].replace(",","")
        engineCapacity.append(engineCCText)


    #find image URL
#     for pictures in soup.find_all('img', {'title': 'more details'}):
    for pictures in soup.find_all('img', {'title': ['more details','no image available']}):
        src = pictures.get('src')
        images.append(src)

    #put all the data into a dataframe    
    for i in range(0, len(desc)):
        data = {
            "Ask Price":price[i],
            "Car Model":desc[i],
            "Brand":brands[i],
            "Depreciation":depreciationValue[i],
            "Registration Date":registrationDate[i],
            "Category":category[i],
            "Mileage":mileage[i],
            "Engine Capacity":engineCapacity[i],
            "Images":images[i]
        }
        dict_car.append(data)

    df = pd.DataFrame(dict_car)
    return df

finaldf = pd.DataFrame(columns=["Ask Price","Car Model","Brand","Depreciation","Registration Date","Category",
                               "Mileage","Engine Capacity","Images"])


for brand in carBrands:
    for i in range(0, 500, 100):
        pagination = str(i)
        link = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + pagination + "&RPG=100" +"&MOD=" + brand +"&AVL=0"
        webpage = urlopen(link).read()
        sleep(random.uniform(1,2))
        df = scraping(webpage)
        finaldf = pd.concat([finaldf, df], axis=0)

    finaldf.reset_index(drop=True, inplace=True)


In [11]:
finaldf

,Ask Price,Car Model,Brand,Depreciation,Registration Date,Category,Mileage,Engine Capacity,Images
0,58800,A3 Sportback 1.4A TFSI S-tronic Ambiente,Audi,12310,08-Aug-2015,Hatchback,62000,1395,https://i.i-sgcm.com/cars_used/202107/1014560_...
1,100900,A6 1.8A TFSI Ultra S-tronic,Audi,14840,14-Dec-2016,Luxury,46000,1798,https://i.i-sgcm.com/cars_used/202108/1019249_...
2,82888,A7 Sportback 2.8A FSI Quattro S-tronic,Audi,18130,31-Mar-2014,Luxury,111000,2773,https://i.i-sgcm.com/cars_used/202106/1003948_...
3,178800,R8 4.2A FSI Quattro R-tronic (COE till 11/2028),Audi,24500,21-Jan-2009,Sports,84241,4163,https://i.i-sgcm.com/cars_used/202105/997153_s...
4,81000,A4 1.4A TFSI S-tronic,Audi,12990,26-Oct-2016,Luxury,100000,1395,https://i.i-sgcm.com/cars_used/202107/1016975_...
...,...,...,...,...,...,...,...,...,...
1315,0,F430 Spider (COE till 06/2030),Ferrari,N.A,03-Jun-2010,Sports,0,4308,https://i.i-sgcm.com/cars_used/202103/980364_s...
1316,0,California 4.3A (COE till 02/2030),Ferrari,N.A,16-Jul-2010,Sports,0,4297,https://i.i-sgcm.com/cars_used/202104/985740_s...
1317,0,458 Speciale,Ferrari,N.A,18-Sep-2014,Sports,0,4497,https://i.i-sgcm.com/cars_used/202101/959536_s...
1318,0,488 GTB,Ferrari,N.A,26-Jul-2017,Sports,0,3902,https://i.i-sgcm.com/cars_used/202103/978078_s...


In [4]:
#scrap for specific brand

linkOpel = "https://www.sgcarmart.com/used_cars/listing.php?MOD=opel&AVL=2&RPG=100"
webpage = urlopen(linkOpel).read()
df = scraping(webpage)
finaldf = pd.concat([finaldf, df], axis=0)

In [5]:
finaldf

,Ask Price,Car Model,Brand,Depreciation,Registration Date,Category,Mileage,Engine Capacity,Images
0,75888,Levorg 1.6A GT-S,Subaru,13760,29-Feb-2016,Stationwagon,55000,1600,https://i.i-sgcm.com/cars_used/202103/979492_s...
1,69800,Accord 2.4A,Honda,12360,27-Oct-2015,Luxury,83500,2356,https://i.i-sgcm.com/cars_used/202108/1023091_...
2,12888,Corolla Axio 1.5A X (COE till 02/2023),Toyota,8620,13-Feb-2008,Sedan,0,1496,https://i.i-sgcm.com/cars_used/202108/1023090_...
3,68800,Vellfire 2.4A Z G-Edition Moonroof,Toyota,17500,14-Mar-2014,MPV,197000,2362,https://i.i-sgcm.com/cars_used/202108/1023089_...
4,35000,C-Class C180K,Mercedes-Benz,32180,29-Mar-2012,Luxury,75000,1597,https://i.i-sgcm.com/cars_used/202108/1023079_...
...,...,...,...,...,...,...,...,...,...
57,107600,Crossland 1.2A X Turbo (OPC),Opel,11120,26-Jan-2021,SUV,0,1199,https://i.i-sgcm.com/cars_used/202106/1000886_...
58,52000,Astra 1.0A Turbo,Opel,8600,17-May-2017,Hatchback,0,999,https://i.i-sgcm.com/cars_used/202105/999598_s...
59,49800,Meriva 1.4A Turbo,Opel,8560,06-Jun-2016,Hatchback,118000,1364,https://i.i-sgcm.com/cars_used/202106/1005951_...
60,69800,Astra Sports Tourer 1.0A IntelliLux LED (OPC),Opel,8930,08-Mar-2018,Stationwagon,35000,999,https://i.i-sgcm.com/cars_used/202104/985538_s...


In [15]:
indexNames = finaldf[finaldf['Depreciation'] == 'N.A'].index
finaldf.drop(indexNames, inplace=True)

indexName2 = finaldf[finaldf['Ask Price'] == 0].index
finaldf.drop(indexName2, inplace=True)

In [16]:
mileidx = finaldf[finaldf['Mileage'] == 0].index
finaldf.drop(mileidx, inplace=True)

ccidx = finaldf[finaldf['Engine Capacity'] == '-'].index
finaldf.drop(ccidx, inplace=True)

In [17]:
finaldf
finaldf.to_csv('cars.csv', index=False, header=True)